Librerías del Proyecto

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from nltk.tokenize import word_tokenize
import os
import re
import string
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
nltk.download('stopwords')
 
# Importing libraries necessary for Model Building and Training
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
 
import warnings
warnings.filterwarnings('ignore')

import mailbox
from email import policy
from email.parser import BytesParser

In [ ]:
archivo_correo = mailbox.mbox(os.getcwd() + r'\easy_ham\0001.ea7e79d3153e7469e7a9c3e0af6a357e')

with open(os.getcwd() + r'\easy_ham\0001.ea7e79d3153e7469e7a9c3e0af6a357e', 'rb') as fp:
    archivo_correo_2 = BytesParser(policy=policy.default).parse(fp)

archivo_correo_2.get_payload()


In [ ]:

for mensaje in archivo_correo:
    print(mensaje.is_multipart())
    cuerpo = mensaje.get_payload(decode=True).decode(errors='replace')
cuerpo

Abriendo las carpetas de spam y no spam:

In [ ]:
os.getcwd()

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
            if c == '<' and not quote:
                tag = True
            elif c == '>' and not quote:
                tag = False
            elif (c == '"' or c == "'") and tag:
                quote = not quote
            elif not tag:
                out = out + c

    return out

In [67]:
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

In [57]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

In [61]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://S+|www.S+')
    return url_pattern.sub(r'', text)

In [64]:
def remove_numbers(text):
    return ''.join(c for c in text if not c.isdigit())

In [68]:
def remove_stopwords(text):
    stop_words = stopwords.words('english')
 
    imp_words = []
 
    # Storing the important words
    for word in str(text).split():
        word = word.lower()
 
        if word not in stop_words:
            imp_words.append(word)
 
    output = " ".join(imp_words)
 
    return output

In [ ]:
def email_parser(mbox, carp):
    message = {}
    for mensaje in mbox:
        cuerpo = ""
        if mensaje.is_multipart():
            # Si el correo tiene varias partes, buscar la parte de texto/html
            # message['isMultipart'] = True
            for parte in mensaje.get_payload():
                if parte.get_content_type() == 'text/plain':
                    cuerpo = parte.get_payload(decode=True).decode(errors='replace')
                    #cuerpo += cuerpo.translate(str.maketrans('', '', string.punctuation))
                if parte.get_content_type() == 'text/html':
                    cuerpo = parte.get_payload(decode=True).decode(errors='replace')
                    #cuerpo += remove_html_markup(cuerpo)
        else:
            # message['isMultipart'] = False
            # Si el correo es de una sola parte, obtener el cuerpo directamente
            if mensaje.get_content_type() == 'text/plain':
                char_set = mensaje.get_charset()
                cuerpo = mensaje.get_payload(decode=True).decode(errors='replace')
                #cuerpo = cuerpo.translate(str.maketrans('', '', string.punctuation))
            if mensaje.get_content_type() == 'text/html':
                char_set = mensaje.get_charset()
                cuerpo = mensaje.get_payload(decode=True).decode(errors='replace')
                #cuerpo = remove_html_markup(cuerpo)
        message['message'] = cuerpo
        message['type'] = carp

        return message

In [ ]:
carpeta = ["spam", "easy_ham"]  # Ruta de la carpeta que deseas leer
message_list = []
for carp in carpeta:
    # Obtener la lista de archivos en la carpeta
    lista_archivos = [f'{os.path.join(os.getcwd(), carp)}\\{x}' for x in os.listdir(os.path.join(os.getcwd(), carp))]
    #print(lista_archivos)
    # Leer cada archivo de la lista
    for archivo in lista_archivos:
        if os.path.isfile(archivo):  # Verificar que sea un archivo
            # Abrir el dataset de correos
            mbox = mailbox.mbox(archivo)
            # Recorrer cada correo en el dataset
            message = email_parser(mbox, carp)
            if message == None:
                print(archivo)
            else:
                
                message_list.append(message)

In [69]:
for x in message_list:
    x["message"] = re.sub(r"[^\x00-\x7F]+", "", str(x["message"]))
    x["message"] = x["message"].replace("\n", " ").replace("\t", " ").replace("\\u", " ").replace("\r", " ").lower()
    x["message"] = cleanhtml(x["message"])
    x["message"] = remove_html_markup(x["message"])
    x["message"] = remove_urls(x["message"])
    x["message"] = remove_punctuation(x["message"])
    x["message"] = remove_numbers(x["message"])
    x["message"] = remove_stopwords(x["message"])

In [70]:
with open('messages.csv', 'w') as f:
    for x in message_list:
        f.write(f'{x["type"]},{x["message"]}\n')